## Transforms

Raster transformations are the operations that maps pixels of a source raster to a destination raster with an associated coordinate reference systems. The CRSs of source and destination raster may or may not be different.

To perform transformations we will use a Landsat-8 dataset in this lab. 
It is publicly accessible at https://landsat-pds.s3.amazonaws.com/L8/042/034/LC80420342013156LGN00/index.html

More information on Landsat-8 can be found [here](https://landsat.usgs.gov/landsat-collections). 
We will be using two bands of the scene, red and near-infrared for the rest of the labs. 

##### Obtaining the red and near-infrared bands of Landsat-8 collection

In [ ]:
###############-----------------------------
### Obtaining red and near-infrared bands
############## -----------------------------

import urllib.request
import shutil
from pathlib import Path

RED_BAND_URL = 'https://landsat-pds.s3.amazonaws.com/L8/042/034/LC80420342013156LGN00/LC80420342013156LGN00_B4.TIF'
local_file_name = 'redband.TIF'

file_path = Path('../temp/')
file_path /= local_file_name

with urllib.request.urlopen(RED_BAND_URL) as response, file_path.open(mode='w+b') as outfile:
    shutil.copyfileobj(response, outfile)

NIR_BAND_URL = 'https://landsat-pds.s3.amazonaws.com/L8/042/034/LC80420342013156LGN00/LC80420342013156LGN00_B5.TIF'
local_file_name = 'nearIRband.TIF'

file_path = Path('../temp/')
file_path /= local_file_name

with urllib.request.urlopen(NIR_BAND_URL) as response, file_path.open(mode='w+b') as outfile:
    shutil.copyfileobj(response, outfile)

##### Load raster data files

Let's load the raster files that we obtained in the previous step and visualize them.

In [ ]:
###############-----------------------------
### Loading
############## -----------------------------

import rasterio
import numpy as np
from pprint import pprint
import time

# start = time.time()

with rasterio.open('../temp/redband.TIF') as src:
    redband = src.read()

with rasterio.open('../temp/nearIRband.TIF') as src:
    nirband = src.read()
    
# end = time.time()
# print("It took {} milli-seconds to load the raster files".format((end-start)*1000))

In [ ]:
###############-----------------------------
### Visualizing
############## -----------------------------

from rasterio.plot import show
import matplotlib.pyplot as plt

%matplotlib inline

# start = time.time()

fig = plt.figure(figsize= (6,6))
rasterio.plot.show(redband)

fig = plt.figure(figsize= (6,6))
rasterio.plot.show(nirband)
# end = time.time()
# print("It took {} milli-seconds to display the raster files".format((end-start)*1000))

Your images should look like below.

![Red_NIR_bands_original.png MISSING](../images/Red_NIR_bands_original.png)

---


Looks like it took a good amount of time to load and visualize rasters. Uncomment the lines
``` python
#start = time.time()
........
#end = time.time()
#print("...".format((end-start)*1000))
```

in the above to cells to measure the time in milli-seconds and record the time.

### Why did it take so long? 

To understand the reason, let's check the size of each raster image.

In [ ]:
print(redband.shape)
print(nirband.shape)

Now you see the reason, the images are of size (1&times;7531&times;7711, i.e., 58 Megapixels), which is why it took so long to render the images. 
It takes a long time to perfom any operation on such large images. 

So, the transformation that we will be learning in this lab is to reduce the size of image, 
while maintaining the resemblance with the original image. 
This is the **downsampling** transformation.

#### Downsampling
Downsampling is the process of converting high-resolution rasterdata (images) into lower-resolution images.
The following code presents the simplest way to downsample raster images using `rasterio`. 
As the images are of size (1&times;7531&times;7711), we shall reduce it to (1&times;753&times;771)

In [ ]:
import rasterio
import numpy as np

# Create an empty NumPy ND-array of the specified size and with the Unsigned Integer, 16-bit datatype
redband = np.empty(shape=(1,753,771), dtype = np.uint16)
nirband = np.empty(shape=(1,753,771), dtype = np.uint16)
with rasterio.open('../temp/redband.TIF') as src:
    # Note, we provide a container to receive the data
    src.read(out = redband)

with rasterio.open('../temp/nearIRband.TIF') as src:
    # Note, we provide a container to receive the data
    src.read(out= nirband)

To get an idea of performance gain, let's `time` the visualization code.

In [ ]:
from rasterio.plot import show
import matplotlib.pyplot as plt

%matplotlib inline

start = time.time()

fig = plt.figure(figsize= (6,6))
rasterio.plot.show(redband)

fig = plt.figure(figsize=(6,6))
rasterio.plot.show(nirband)

end = time.time()
print("It took {} milli-seconds to display the raster files".format((end-start)*1000))

From now on, we would be dealing with the downsampled images. 
Here we present another way of downsampling raster data, and save the downsampled images to the `temp` folder.

In the code, we create a new empty array `downsampled` to store the downsampled version of the original image, 
set its height and width to 1/10<sup>th</sup> of the height and widths of the original image. 

_Note: The type of variables `redband` and `downsampled` has to match. In our case, both of them are of `uint16`._

In [ ]:
from rasterio import Affine
from rasterio.warp import reproject,RESAMPLING
import numpy as np
%matplotlib inline

src = rasterio.open('../temp/redband.TIF')
redband = src.read()
height = round(redband.shape[1]/10)
width = round(redband.shape[2]/10)
downsampled = np.empty(shape = (redband.shape[0], height, width), dtype = np.uint16)


Before we proceed with the downsampling, let's inspect and understand the transform property of source raster.

In [ ]:
print(src.transform)

The transform is a tuple of 6 numbers, 
the first and fourth corresponds to the world coordinates of top left corner pixel of the raster data. 
Second and sixth represents the resolution. 
In this case, the raster has a resolution of 30m in height and width.

**Note:** The negative 30 is because in Image coordinates, Y gets larger as you move from Top-Left downward.
However, in geographic and mathematical coordinate spaces, the Y gets larger as you move upward.

When we try to reduce the size of the raster data, 
the world coordinates of the top left corner stays the same, only the resolution changes. 
As we are reducing the size by 10 in both `x` and `y` directions, the resolution has to increase by 10,
i.e., the downsampled image should have a resolution of 300m in `x` and `y` directions.

In [ ]:
dst_transform = src.transform.copy()

dst_transform[1] = dst_transform[1]*10
dst_transform[5] = dst_transform[5]*10

print(dst_transform)

We use [`reproject()`](https://mapbox.s3.amazonaws.com/playground/perrygeo/rasterio-docs/reproject.html) method to perform this transformation and visualize the downsampled raster.

In [ ]:
reproject(
    redband, 
    downsampled,
    src_transform = src.transform,
    src_crs = src.crs,
    dst_transform = dst_transform,
    dst_crs = src.crs,
    resampling = RESAMPLING.nearest)

fig = plt.figure(figsize=(6,6))
rasterio.plot.show(downsampled)

We can then write the downsampled raster image to disk.

**Note:** _The downsampled version of raster images are used in next labs._

In [ ]:
kwargs = src.meta.copy()
kwargs.update({
    'crs': src.crs,
    'transform': dst_transform,
    'width' : width,
    'height': height
})

with rasterio.open('../temp/redband_downsampled.TIF','w', **kwargs) as dst:
    reproject(
        source = rasterio.band(src,1), 
        destination = rasterio.band(dst,1),
        src_transform = src.transform,
        src_crs = src.crs,
        dst_transform = dst_transform,
        dst_crs = src.crs,
        resampling = RESAMPLING.nearest)
    

Load and visualize the downsampled raster to make sure it resembles the original raster.

In [ ]:
with rasterio.open('../temp/redband_downsampled.TIF') as src:
    fig = plt.figure(figsize = (6,6))
    rasterio.plot.show(src.read())

### Down sample the `nearIRband.TIF` following the above procedure and save it as `nearIRband_downsampled.TIF`.
****

In [ ]:
from rasterio import Affine
from rasterio.warp import reproject,RESAMPLING
import numpy as np
%matplotlib inline

src = rasterio.open('../temp/nearIRband.TIF')
nirband = src.read()
height = round(nirband.shape[1]/10)  
width = round(nirband.shape[2]/10)   
downsampled = np.empty(shape = (nirband.shape[0], height, width), dtype = np.uint16)


In [ ]:
dst_transform = src.transform.copy()

dst_transform[1] = dst_transform[1]*10
dst_transform[5] = dst_transform[5]*10


In [ ]:
kwargs = src.meta.copy()
kwargs.update({
    'crs': src.crs,
    'transform': dst_transform,
    'width' : width,
    'height': height
})

with rasterio.open('../temp/nearIRband_downsampled.TIF','w', **kwargs) as dst:
    reproject(
        source = rasterio.band(src,1), 
        destination = rasterio.band(dst,1),
        src_transform = src.transform,
        src_crs = src.crs,
        dst_transform = dst_transform,
        dst_crs = src.crs,
        resampling = RESAMPLING.nearest)
    

In [ ]:
with rasterio.open('../temp/nearIRband_downsampled.TIF') as src:
    fig = plt.figure(figsize = (6,6))
    rasterio.plot.show(src.read())

****
# Save your Notebook
## Then, File > Close and Halt